### 배포 - npm build 후에 배포

In [5]:
import paramiko

import os
def get_file_list(path):
    return [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

hostname = "maxy.iptime.org"
# hostname = "192.168.10.177"
username = "songs"
password = "thdghqkr74"
port = 8022
remote_path = "/mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back"
# SSH 클라이언트 객체 생성
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [6]:

def check_remote_dir(sftp, remote_path):
    """원격 경로가 없으면 계층적으로 생성"""
    dirs = remote_path.strip('/').split('/')
    path = ''
    for d in dirs:
        path += '/' + d
        try:
            sftp.stat(path)  # 존재 여부 확인
        except FileNotFoundError:
            sftp.mkdir(path)
            

def FileUpload(sftp,SourceFile,TargetFile):
    try:
        with sftp.open(TargetFile, "wb") as remote_file, open(SourceFile, "rb") as local_file:
            chunk_size = 1024 * 1024  # 32KB씩 다운로드
            downloaded_size = 0
            while True:
                data = local_file.read(chunk_size)
                if not data:
                    break
                remote_file.write(data)
                downloaded_size += len(data)
                print(f"업로드중: {SourceFile} -> {downloaded_size}")
        print(f"완료: {TargetFile}")
    except Exception as e:
        print(f"FileUpload 오류 발생: {SourceFile}-{TargetFile}-{e}")
        
        
def FolderUpdate(sftp,SourceFolder,TargetFolder):
    try:
        # flist = get_file_list(f"{SourceFolder}")
        flist = os.listdir(f"{SourceFolder}")
        for file in flist:
            if file == "." or file == "..":
                continue
            if os.path.isdir(f"{SourceFolder}/{file}"):
                check_remote_dir(sftp,f"{TargetFolder}/{file}")
                FolderUpdate(sftp,f"{SourceFolder}/{file}",f"{TargetFolder}/{file}")
                continue
            SourceFile = f"{SourceFolder}/{file}"
            TargetFile = f"{TargetFolder}/{file}"
            FileUpload(sftp,SourceFile,TargetFile)
    except Exception as e:
        print(f"FolderUpdate 오류 발생: {SourceFolder}-{TargetFolder}-{e}")

In [7]:
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh_client.connect(hostname, port, username, password)
    TargetPath = f"{remote_path}"
    SourcePath = f"."
    # SFTP 클라이언트 생성
    sftp = ssh_client.open_sftp()
    FileUploadDir = ["Data","models","PFunnelAnalysis","playload","PManagement","PPerformanceAnalysis","PTotalAnalysis","Waterfall","widget","ClickHouseComm"]
    for folder in FileUploadDir:
        check_remote_dir(sftp,f"{TargetPath}/{folder}")
        FolderUpdate(sftp,f"{SourcePath}/{folder}",f"{TargetPath}/{folder}")    
    UploadFiles = [
        (f"{SourcePath}/apiserver.py",f"{TargetPath}/apiserver.py")
    ]
    for SourceFile,TargetFile in UploadFiles:
        FileUpload(sftp,SourceFile,TargetFile)
    sftp.close()
except Exception as e:
    print(f"오류 발생: {e}")
finally:
    ssh_client.close()

업로드중: ./Data/marketing.funnel_info.csv -> 2806
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/marketing.funnel_info.csv
업로드중: ./Data/group_app_map.csv -> 51
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/group_app_map.csv
업로드중: ./Data/marketing.group_by_id.csv -> 48
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/marketing.group_by_id.csv
업로드중: ./Data/marketing.condition_field.csv -> 1070
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/marketing.condition_field.csv
업로드중: ./Data/marketing.condition_category.csv -> 255
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/marketing.condition_category.csv
업로드중: ./Data/marketing.discover_group.csv -> 75
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/marketing.discover_group.csv
업로드중: ./Data/group_user_map.csv -> 44
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/Data/group_user_map.csv
업로드중: ./Data/marketing.group.csv -> 2992
완료: /mnt/HDII/Projects/ma

업로드중: ./PFunnelAnalysis/__pycache__/routes.cpython-312.pyc -> 70735
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/PFunnelAnalysis/__pycache__/routes.cpython-312.pyc
업로드중: ./PFunnelAnalysis/__pycache__/funnelModels.cpython-312.pyc -> 8407
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/PFunnelAnalysis/__pycache__/funnelModels.cpython-312.pyc
업로드중: ./PFunnelAnalysis/__pycache__/__init__.cpython-312.pyc -> 236
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/PFunnelAnalysis/__pycache__/__init__.cpython-312.pyc
업로드중: ./PFunnelAnalysis/data_filter.json -> 486
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/PFunnelAnalysis/data_filter.json
업로드중: ./PFunnelAnalysis/routes.py -> 61687
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/PFunnelAnalysis/routes.py
업로드중: ./playload/areaDistributionPayload.py -> 6814
완료: /mnt/HDII/Projects/maxy/compose/maxy-react/maxy_back/playload/areaDistributionPayload.py
업로드중: ./playload/accessibilityPayload.py -> 5599
완

In [8]:
# flist = get_file_list(f".next")
# os.listdir(path=".next")